Authors: Hofmarcher

Date: 20-03-2023

---

This file is part of the "Deep Reinforcement Learning" lecture material. The following copyright statement applies to all code within this file.

Copyright statement:
This material, no matter whether in printed or electronic form, may be used for personal and non-commercial educational use only. Any reproduction of this manuscript, no matter whether as a whole or in parts, no matter whether in printed or in electronic form, requires explicit prior acceptance of the authors.

## Enable GPU Acceleration

---
Before you start exploring this notebook make sure that GPU support is enabled.
To enable the GPU backend for your notebook, go to **Edit** → **Notebook Settings** and set **Hardware accelerator** to **GPU**. 

---


# Imports

Install Gymnasium and dependencies to render the environments

In [ ]:
! pip install wandb 1> /dev/null
# RELOAD KERNEL after installing wandb!

In [ ]:
!apt update 1> /dev/null

!apt install -y \
  xvfb \
  x11-utils \
  python-opengl \
  ffmpeg \
  swig \
1> /dev/null

!pip install \
  gymnasium==0.27.1 \
  gymnasium[box2d] \
  pyvirtualdisplay \
  imageio-ffmpeg \
  moviepy==1.0.3 \
  onnx==1.13.0 \
  onnx2pytorch==0.4.1 \
1> /dev/null

In [ ]:
%matplotlib inline

# Auxiliary Python imports
import os
import math
import io
import base64
import random
import shutil
from time import time, strftime
from glob import glob
from tqdm import tqdm
import numpy as np

# Pytorch
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
import onnx
from onnx2pytorch import ConvertModel

# Environment import and set logger level to display error only
import gymnasium as gym
from gymnasium.spaces import Box
from gymnasium import logger as gymlogger
from gymnasium.wrappers import RecordVideo
gymlogger.set_level(gym.logger.ERROR)

# Plotting and notebook imports
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, clear_output
from IPython import display

# Select device for training

By default we train on GPU if one is available, otherwise we fall back to the CPU.
If you want to always use the CPU change accordingly.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: " + str(device))

In [ ]:
# ! nvidia-smi

# Setup Google Drive mount to store your results

In [ ]:
use_google_drive = False
if use_google_drive:
    from google.colab import drive
    drive.mount('/content/drive')

# Download Dataset and Expert model

In [ ]:
import os

# Download training and validation datasets

if not os.path.exists('expert.onnx'):
    !wget --no-check-certificate 'https://cloud.ml.jku.at/s/citYJKPgmAGrHGy/download' -O expert.onnx

if not os.path.exists('train.zip'):
    !wget --no-check-certificate 'https://cloud.ml.jku.at/s/yJ2ZsfqTos3Jn9y/download' -O train.zip

if not os.path.exists('val.zip'):
    !wget --no-check-certificate 'https://cloud.ml.jku.at/s/3DxHLiqxTddepp8/download' -O val.zip

# Unzip datasets
!unzip -q -o train.zip
!unzip -q -o val.zip

# Auxiliary Methods

The following cell contains classes and functions to provide some functionality for logging, plotting and exporting your model in the format required by the submission server.
You are free to use your own logging framework if you wish (such as tensorboard or Weights & Biases).
The logger is a very simple implementation of a CSV-file based logger.
Additionally it creates a folder for each run with subfolders for model files, logs and videos.

In [ ]:
class Logger():
    def __init__(self, logdir, params=None):
        self.basepath = os.path.join(logdir, strftime("%Y-%m-%dT%H-%M-%S"))
        os.makedirs(self.basepath, exist_ok=True)
        os.makedirs(self.log_dir, exist_ok=True)
        if params is not None and os.path.exists(params):
            shutil.copyfile(params, os.path.join(self.basepath, "params.pkl"))
        self.log_dict = {}
        self.dump_idx = {}

    @property
    def param_file(self):
        return os.path.join(self.basepath, "params.pkl")

    @property
    def onnx_file(self):
        return os.path.join(self.basepath, "model.onnx")

    @property
    def video_dir(self):
        return os.path.join(self.basepath, "videos")
    
    @property
    def log_dir(self):
        return os.path.join(self.basepath, "logs")

    def log(self, name, value):
        if name not in self.log_dict:
            self.log_dict[name] = []
            self.dump_idx[name] = -1
        self.log_dict[name].append((len(self.log_dict[name]), time(), value))
    
    def get_values(self, name):
        if name in self.log_dict:
            return [x[2] for x in self.log_dict[name]]
        return None
    
    def dump(self):
        for name, rows in self.log_dict.items():
            with open(os.path.join(self.log_dir, name + ".log"), "a") as f:
                for i, row in enumerate(rows):
                    if i > self.dump_idx[name]:
                        f.write(",".join([str(x) for x in row]) + "\n")
                        self.dump_idx[name] = i


def plot_metrics(logger):
    train_loss  = logger.get_values("training_loss")
    train_entropy  = logger.get_values("training_entropy")
    val_loss = logger.get_values("validation_loss")
    val_acc = logger.get_values("validation_accuracy")
    
    fig = plt.figure(figsize=(15,5))
    ax1 = fig.add_subplot(131, label="train")
    ax2 = fig.add_subplot(131, label="val",frame_on=False)
    ax4 = fig.add_subplot(132, label="entropy")
    ax3 = fig.add_subplot(133, label="acc")

    ax1.plot(train_loss, color="C0")
    ax1.set_ylabel("Loss")
    ax1.set_xlabel("Update (Training)", color="C0")        
    ax1.xaxis.grid(False)  
    ax1.set_ylim((0,4))

    ax2.plot(val_loss, color="C1")
    ax2.xaxis.tick_top()
    ax2.yaxis.tick_right()
    ax2.set_xlabel('Epoch (Validation)', color="C1")     
    ax2.xaxis.set_label_position('top')     
    ax2.xaxis.grid(False)
    ax2.get_yaxis().set_visible(False)
    ax2.set_ylim((0,4))

    ax4.plot(train_entropy, color="C3")    
    ax4.set_xlabel('Update (Training)', color="black")     
    ax4.set_ylabel("Entropy", color="C3")
    ax4.tick_params(axis='x', colors="black")
    ax4.tick_params(axis='y', colors="black")
    ax4.xaxis.grid(False)

    ax3.plot(val_acc, color="C2")
    ax3.set_xlabel("Epoch (Validation)", color="black")
    ax3.set_ylabel("Accuracy", color="C2")
    ax3.tick_params(axis='x', colors="black")
    ax3.tick_params(axis='y', colors="black")
    ax3.xaxis.grid(False)
    ax3.set_ylim((0,1))

    fig.tight_layout(pad=2.0)
    plt.show()
    
"""
Utility functions to enable video recording of gym environment and displaying it
"""
def show_video_from_folder(video_dir):
    mp4list = glob(f'{video_dir}/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display.display(HTML(data='''<video alt="test" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")

def show_video_from_file(file):
    video = io.open(file, 'r+b').read()
    encoded = base64.b64encode(video)
    display.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
              </video>'''.format(encoded.decode('ascii'))))

def save_as_onnx(torch_model, sample_input, model_path):
    torch.onnx.export(torch_model,             # model being run
                    sample_input,              # model input (or a tuple for multiple inputs)
                    f=model_path,              # where to save the model (can be a file or file-like object)
                    export_params=True,        # store the trained parameter weights inside the model file
                    opset_version=17,          # the ONNX version to export the model to - see https://github.com/microsoft/onnxruntime/blob/master/docs/Versioning.md
                    do_constant_folding=True,  # whether to execute constant folding for optimization
                    )

# Dataset

Use this dataset class to load the provided demonstrations. Furthermore, this dataset has functionality to add new samples to the dataset which you will need for implementing the DAgger algorithm.

In [ ]:
class DemonstrationDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.files = sorted(glob(f"{data_dir}/*.npz"))
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        data = np.load(self.files[idx])
        state = data["state"].astype(np.float32)
        if state.ndim == 2:
            state = np.expand_dims(state, axis=0)
        action = data["action"]
        return state / 255.0, action.item()
    
    def append_one(self, state, action):
        offset = len(self) + 1
        filename = f"{self.data_dir}/{offset:08}.npz"
        *remaining, w, h = state.shape
        if np.prod(remaining) != 1:
            raise ValueError(f"unexpected state shape when adding to dataset: {state.shape}")
        if action.size != 1:
            raise ValueError(f"unexpected action shape when adding to dataset: {state.shape}")
        np.savez_compressed(
            filename,
            state=state.reshape(w, h),
            action=action.reshape(()).astype(np.int32)
        )
        self.files.append(filename)

    def append_more(self, states, actions):
        for state, action in zip(states, actions):
            self.append_one(state, actions)
    
    def append_iterable(self, states_and_actions):
        for state, action in states_and_actions:
            self.append_one(state, action)
            

# Inspect data

It is always a good idea to take a look at the data when you start working with a new dataset. Feel free to investigate the dataset further on your own.

In [ ]:
# Action Statistics
dataset = DemonstrationDataset("train")
print("Number of samples: {}".format(len(dataset)));

In [ ]:
# Action mapping from gymnasium.farama.org
action_mapping = {
    0: "do nothing",
    1: "steer left",
    2: "steer right",
    3: "gas",
    4: "brake"
}

# Visualize random frames
idx = np.random.randint(len(dataset))
state, action = dataset[idx]
# store a single frame as we need it later for exporting an ONNX model (it needs a sample of the input for the export)
sample_state = torch.Tensor(state).unsqueeze(0).to(device)
# Display the sample
print(f"Action: {action_mapping[action]}")
plt.axis("off")
plt.imshow(state[0]);

In [ ]:
# release memory
del dataset

# Define Policy Network

You need to design a neural network architecture that is capable of mapping a state to an action.
The input is a single image with the following properties:
- Resolution of 84x84 pixels
- Grayscale (meaning a single channel as opposed to three channels of an RGB image)
- The values of each pixel should be between 0 and 1

The output of the network should be one unit per possible action, as our environment has 5 actions that results in 5 output units.
Your network must implement the forward function in order to be compatible with the evaluation script.

In [ ]:
import torchvision
from __future__ import annotations


def set_submodule(module: torch.nn.Module, submodule_path: str, submodule: torch.nn.Module) -> None:
    *parent_path, attr_name = submodule_path.split(".")
    parent = module
    for child in parent_path:
        parent = getattr(parent, child)
    setattr(parent, attr_name, submodule)


class PolicyNetwork(nn.Module):
    """
    Policy network is compatible with any CNN architecture as long as:
    1) One of the conv layers determines how many channels it can process
    2) One of the linear layers determines the number of output neurons
    
    This should be enough to support all standard architectures

    >>> policy_nn = PolicyNetwork(
    ...     actions=5,
    ...     architecture="efficientnet_b0",
    ...     input_conv="features.0.0",
    ...     output_lin="classifier.1",
    ...     pretrained_weights=None, # "IMAGENET1K_V1"
    ... )
    >>> inputs = torch.rand(size=(16, 1, 64, 64))
    >>> policy_nn(inputs).shape
    torch.Size([16, 5])

    """

    def __init__(
        self,
        actions: int | dict[int, str],
        architecture: str,
        input_conv: str,
        output_lin: str,
        pretrained_weights = None,
        seed: int = 42,
    ) -> None:
        super(PolicyNetwork, self).__init__()
        if isinstance(actions, int):
            self.n_units_out = actions
            self.actions = {i: f"action_{i}" for i in range(actions)}
        else:
            self.n_units_out = len(actions)
            self.actions = actions
        self.architecture = architecture
        self.input_conv = input_conv
        self.output_lin = output_lin
        self.init(pretrained_weights, seed)

    def init(self, pretrained_weights = None, seed: int = None) -> None:
        """
        Initializes the model parameters, either randomly or with pretrained weights.
        Patches the architecture to have desired input and output shape.
        """
        if seed is not None:
            torch.random.manual_seed(seed)
        self.cnn = torchvision.models.get_model(self.architecture, weights=pretrained_weights)
        self._patch_input_shape()
        self._patch_output_shape()

    def _patch_input_shape(self) -> None:
        "Make the architecture accept a single (grayscale) channel"
        old_in_conv = self.cnn.get_submodule(self.input_conv)
        assert isinstance(old_in_conv, torch.nn.Conv2d)
        new_in_conv = torch.nn.Conv2d(
            in_channels = 1, 
            out_channels = old_in_conv.out_channels,
            kernel_size = old_in_conv.kernel_size,
            stride = old_in_conv.stride,
            padding = old_in_conv.padding,
            bias = old_in_conv.bias is not None,
        )
        set_submodule(self.cnn, self.input_conv, new_in_conv)

    def _patch_output_shape(self) -> None:
        "Make the architecture output the correct shape"
        old_out_lin = self.cnn.get_submodule(self.output_lin)
        assert isinstance(old_out_lin, torch.nn.Linear)
        new_out_lin = torch.nn.Linear(
            in_features = old_out_lin.in_features,
            out_features = self.n_units_out,
            bias = old_out_lin.bias is not None,
        )
        set_submodule(self.cnn, self.output_lin, new_out_lin)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # select only last channel if there are more
        x = x[:, (-1,), :, :]
        return self.cnn(x)

# Train behavioral cloning policy

Now that you have a Dataset and a network you need to train your network.
With behavioral cloning we want to imitate the behavior of the agent that produced the demonstration dataset as close as possible.
This is basically supervised learning, where you want to minimize the loss of your network on the training and validation sets.

Some tips as to what you need to implement:
- choose the appropriate loss function (think on which kind of problem you are solving)
- choose an optimizer and its hyper-parameters
- optional: use a learning-rate scheduler
- don't forget to evaluate your network on the validation set
- store your model and training progress often so you don't loose progress if your program crashes

In case you use the provided Logger:
- `logger.log("training_loss", <loss-value>)` to log a particular value
- `logger.dump()` to write the current logs to a log file (e.g. after every episode)
- `logger.log_dir`, `logger.param_file`, `logger.onnx_file`, `logger.video_dir` point to files or directories you can use to save files
- you might want to specify your google drive folder as a logdir in order to automatically sync your results
- if you log the metrics specified in the `plot_metrics` function you can use it to visualize your training progress (or take it as a template to plot your own metrics)

In [ ]:
class CropObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        gym.ObservationWrapper.__init__(self, env)
        self.shape = shape
        obs_shape = self.shape + env.observation_space.shape[2:]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def observation(self, observation):
        return observation[:self.shape[0], :self.shape[1]]

    
class RecordState(gym.Wrapper):
    def __init__(self, env: gym.Env, enabled, reset_clean: bool = True):
        gym.Wrapper.__init__(self, env)

        assert env.render_mode is not None
        self.frame_list = []
        self.reset_clean = reset_clean
        self.enabled = enabled

    def step(self, action, **kwargs):
        output = self.env.step(action, **kwargs)
        if self.enabled:
            self.frame_list.append(output[0])
        return output

    def reset(self, *args, **kwargs):
        result = self.env.reset(*args, **kwargs)

        if self.reset_clean:
            self.frame_list = []
        if self.enabled:
            self.frame_list.append(result[0])

        return result

    def render(self):
        frames = self.frame_list
        self.frame_list = []
        return frames
    

class Agent():
    def __init__(self, model, device, supports_batch):
        self.model = model
        self.device = device
        self.supports_batch = supports_batch

    def select_action(self, state):
        if not isinstance(state, torch.Tensor):
            state = torch.tensor(state)

        if not self.supports_batch and state.ndim == 4 and state.shape[0] > 1:
            return np.stack([self.select_action(s) for s in state])

        with torch.no_grad():
            self.model.eval().to(self.device)
            state = state.to(self.device) / 255.0 # rescale
            if len(state.shape) == 3:
              state = state.unsqueeze(0)
            logits = self.model(state)
            if type(logits) is tuple:
                logits = logits[0]
            probs = Categorical(logits=logits)
            selected = probs.sample().cpu().numpy()
            if selected.size == 1:
                return selected.item()
            return selected
        
            
def make_env(seed, capture_video=True, enabled_record_state=False):
    env = gym.make("CarRacing-v2", render_mode="rgb_array", continuous=False)
    env = gym.wrappers.RecordEpisodeStatistics(env)
    if capture_video:
        env = gym.wrappers.RecordVideo(env, logger.video_dir)
            
    env = CropObservation(env, (84, 96))
    env = gym.wrappers.ResizeObservation(env, (84, 84))
    env = gym.wrappers.GrayScaleObservation(env)    
    env = RecordState(env, reset_clean=True, enabled=enabled_record_state)
    env = gym.wrappers.FrameStack(env, 4)
    env.reset(seed=seed)
    env.action_space.seed(seed)
    env.observation_space.seed(seed)
    return env


def run_episode(agent, show_progress=True, capture_video=True, seed=None):
    env = make_env(seed=seed, capture_video=capture_video)
    state, _ = env.reset()
    score = 0
    done = False
    if show_progress:
        progress = tqdm(desc="Score: 0")
        
    while not done:
        action = agent.select_action(state[-1][np.newaxis, ...])
        state, reward, terminated, truncated, _ = env.step(action)
        score += reward
        done = terminated or truncated
        if show_progress:
            progress.update()
            progress.set_description("Score: {:.2f}".format(score))       
    env.close()
    
    if show_progress:
        progress.close()    
    if capture_video:
        show_video_from_folder(logger.video_dir)
    
    return score

In [ ]:
# Specify the google drive mount here if you want to store logs and weights there (and set it up earlier)
logger = Logger("logdir_dagger")
print("Saving state to {}".format(logger.basepath))


In [ ]:
! pip install \
  lovely-tensors==0.1.14 \
  lovely-numpy==0.2.8 \
  lightning==2.0.1 \
1> /dev/null

In [ ]:
import lovely_tensors
import lovely_numpy

def lovely(x):
    "summarizes important tensor properties"
    if isinstance(x, np.ndarray):
        return lovely_numpy.lovely(x)
    return lovely_tensors.lovely(x)

In [ ]:
from typing import Iterable

def measure_scores_parallel(agent: Agent, num_episodes: int, limit_steps: int = None, return_unfinished: bool = False) -> Iterable[float]:
    env = gym.vector.AsyncVectorEnv([
        lambda: make_env(seed=None, capture_video=False)
        for _ in range(num_episodes)
    ])
    
    if limit_steps is None:
        limit_steps = env.get_attr("spec")[0].max_episode_steps
    if limit_steps is None:
        raise ValueError("limit_steps must not be None")

    state, _ = env.reset()
    score = np.zeros(num_episodes)
    done = np.zeros(num_episodes, dtype=bool)

    for i in tqdm(range(limit_steps), "Measuring score - step"):
        action = agent.select_action(state)
        state, reward, terminated, truncated, _ = env.step(action)
        # don't update score of episodes that ended already
        # vector env resets them automatically and starts over
        score[~done] += reward[~done]
        curr_ended = terminated | truncated
        done |= curr_ended
        for idx in curr_ended.nonzero()[0]:
            yield score[idx].item()

        if done.all():
            break
      
    env.close()

    if return_unfinished:
        yield from score[~done].tolist()


In [ ]:
import lightning

class MetricsHistoryKeeper(lightning.Callback):
    def __init__(self):
        super().__init__()
        self.history = []

    def on_validation_epoch_end(self, trainer, pl_module):
        metrics = {}
        for key, value in trainer.callback_metrics.items():
            if isinstance(value, torch.Tensor):
                value = value.cpu().numpy()
            if isinstance(value, np.ndarray) and value.size == 1:
                value = value.item()
            metrics[key] = value
        self.history.append(metrics)

In [ ]:
from __future__ import annotations

import wandb
from typing import Any
from tqdm.auto import tqdm

# Datasets
train_set = DemonstrationDataset("train")
val_set = DemonstrationDataset("val")

# Specify the google drive mount here if you want to store logs and weights there (and set it up earlier)
# You can also choose to use a different logging framework such as tensorboard (not recommended on Colab) or Weights & Biases (highly recommended)
#logger = Logger("logdir")
#print("Saving state to {}".format(logger.basepath))


######################
### YOUR CODE HERE ###
######################


import time
import copy
import pathlib
from typing import Tuple, Dict

import pandas as pd


class LightningPolicyNetwork(lightning.LightningModule):
    
    def __init__(
        self,
        model_config: dict[str, Any],
        num_val_episodes: int,
        limit_val_steps: int | None,
        label_smoothing: float,
        dummy_score_during_sanity_check: float | None,
    ) -> None:

        super().__init__()
        self.save_hyperparameters()
        self.model_config = model_config
        self.num_val_episodes = num_val_episodes
        self.limit_val_steps = limit_val_steps
        self.dummy_score_during_sanity_check = dummy_score_during_sanity_check

        self.model = PolicyNetwork(**model_config, pretrained_weights=None)
        self.loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=label_smoothing)
        self.loss_name = "cross_entropy_loss"

        self.simulation_test_kwargs = None

    def configure_optimizers(self) -> torch.optim.Optimizer:
        return torch.optim.Adam(self.parameters(), lr=1e-4)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

    @staticmethod
    def _entropy(logits: torch.Tensor) -> torch.Tensor:
        return -(logits.softmax(dim=-1) * logits.log_softmax(dim=-1)).sum(dim=-1).mean()

    def training_step(self, batch: Tuple[torch.Tensor, torch.Tensor], batch_idx) -> torch.Tensor:
        state, action = batch
        logits = self(state)
        loss = self.loss_fn(logits, action)
        self.log_dict({
            "train/" + self.loss_name: loss,
            "train/entropy": self._entropy(logits)
        })
        return loss
    
    def validation_step(self, batch: Tuple[torch.Tensor, torch.Tensor], batch_idx) -> torch.Tensor:
        state, action = batch
        logits: torch.Tensor = self(state)
        loss = self.loss_fn(logits, action)
        accuracy = (logits.argmax(dim=-1) == action).float().mean()
        self.log_dict({
            "valid/" + self.loss_name: loss,
            "valid/entropy": self._entropy(logits),
            "valid/accuracy": accuracy,
        })
        return loss

    def test_in_simulation(self, num_episodes: int, limit_steps: int, return_unfinished: bool, log: bool, device = None) -> np.ndarray:
        if device is None:
            device = self.device
        agent = Agent(self.model, device, supports_batch=True)
        scores = measure_scores_parallel(agent, num_episodes, limit_steps, return_unfinished)
        scores = np.array(list(scores))
        if log:
            self.log_simulation_metrics(scores, limit_steps, return_unfinished)
        return scores

    def simulation_metrics(self, scores: np.ndarray) -> dict[str, Any]:
        basic = {
            "num_episodes": float(len(scores)),
            "score_avg": float(scores.mean()),
            "score_std": float(scores.std()),
            "score_min": float(scores.min()),
            "score_max": float(scores.max())
        }
        percentiles = {
            f"score_percentile_{p}": pd.Series(scores).quantile(p / 100)
            for p in [20, 35, 50, 65, 80]
        }
        return {**basic, **percentiles}

    def log_simulation_metrics(self, scores: np.ndarray, limit_steps: int, return_unfinished: bool) -> dict[str, Any]:
        prefix = "simulation" if limit_steps is None else "simulation_limited"

        for logger in self.loggers:
            if not isinstance(logger, lightning.pytorch.loggers.WandbLogger):
                continue
            logger.log_metrics({f"{prefix}/score": wandb.Histogram(scores)})

        self.log_dict({
            f"{prefix}/{name}": value
            for name, value in self.simulation_metrics(scores).items()
        })

        self.log_dict({
            f"{prefix}/limit_steps": -1.0 if limit_steps is None else float(limit_steps),
            f"{prefix}/return_unfinished": 1.0 if return_unfinished else 0.0,
            f"{prefix}/was_sanity_check": 1.0 if self.trainer.sanity_checking else 0.0,
        })

    def on_validation_epoch_end(self) -> None:
        if self.trainer.sanity_checking:
            limit_steps = 10
        else:
            limit_steps = self.limit_val_steps

        return_unfinished = True
        scores = self.test_in_simulation(self.num_val_episodes, limit_steps, return_unfinished, log=False)

        if self.trainer.sanity_checking and self.dummy_score_during_sanity_check is not None:
            scores = np.full(self.num_val_episodes, self.dummy_score_during_sanity_check, dtype=float)
            limit_steps = 0
            return_unfinished = False

        self.log_simulation_metrics(scores, limit_steps, return_unfinished)

    def prepare_testing(self, **kwargs: Any) -> None:
        self.simulation_test_kwargs = kwargs

    def test_step(self, *args: Any, **kwargs: Any) -> None:
        print("test step: IGNORING input")
    
    def on_test_epoch_end(self):
         return self.test_in_simulation(**self.simulation_test_kwargs, log=True)


class MyDataModule(lightning.LightningDataModule):
    def __init__(self, train_set: DemonstrationDataset, val_set: DemonstrationDataset, batch_size: int, num_workers: int, prefetch_factor: int, pin_memory: int):
        super().__init__()
        self.batch_size = batch_size
        self.train_set = train_set
        self.val_set = val_set
        self.num_workers = num_workers
        self.prefetch_factor = prefetch_factor
        self.pin_memory = pin_memory

    def train_dataloader(self):
        print(f"RELOADING TRAIN DATALOADER. dataset size = {len(self.train_set)}")
        return torch.utils.data.DataLoader(
            self.train_set,
            shuffle=True,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            prefetch_factor=self.prefetch_factor,
            pin_memory=self.pin_memory,
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.val_set,
            shuffle=False,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            prefetch_factor=self.prefetch_factor,
            pin_memory=self.pin_memory,
        )


wandb_logger = lightning.pytorch.loggers.WandbLogger(
    project="jku-deep-rl_imitation-learning",
    save_dir="wandb",
)

checkpointing = lightning.pytorch.callbacks.ModelCheckpoint(
    monitor="simulation_limited/score_avg",
    mode="max",
    save_last=True,
    save_top_k=5,
    dirpath=f"checkpoints/{wandb_logger.experiment.name}/",
    filename="step={step}_score-avg={simulation/score_avg:.2f}_score-percentile-30={simulation/score_percentile_30}",
    auto_insert_metric_name=False,
)

metrics_history_keeper = MetricsHistoryKeeper()

model_config = dict(
    actions=action_mapping,
    architecture="efficientnet_b0",
    input_conv="features.0.0",
    output_lin="classifier.1",
)

lightning_model = LightningPolicyNetwork(
    model_config,
    num_val_episodes=8,
    limit_val_steps=500,
    label_smoothing=0.01,
    dummy_score_during_sanity_check=0,
)

wandb_logger.experiment.config.update({
    "dagger_enabled": False,
})

datamodule = MyDataModule(
    train_set=train_set,
    val_set=val_set,
    num_workers=4,
    batch_size=128,
    prefetch_factor=8,
    pin_memory=True,
)

num_batches = len(train_set) / datamodule.batch_size

trainer = lightning.Trainer(
    logger=wandb_logger,
    val_check_interval=int(0.8 * num_batches),
    max_epochs=3,
    precision="16-mixed",
    accelerator="auto",
    callbacks=[
        checkpointing,
        metrics_history_keeper,
    ],
)

In [ ]:
trainer.fit(
    lightning_model,
    datamodule
)

In [ ]:
lightning_model = LightningPolicyNetwork.load_from_checkpoint(checkpointing.best_model_path)

In [ ]:
# If you want to export your model as an ONNX file use the following code as template
# If you use the provided logger you can use this directly
save_as_onnx(
    lightning_model.model.to("cpu"),
    sample_state.to("cpu"),
    pathlib.Path(checkpointing.best_model_path).with_suffix(".onnx"),
)

# Evaluate the agent in the real environment

### Environment and Agent

We provide some wrappers you need in order to get the same states from the environment as in the demonstration dataset.
Additionally the `RecordState` wrapper should be very helpful in collecting new samples for the DAgger algorithm.

## Evaluate behavioral cloning agent

In [ ]:
import pandas as pd

history = pd.DataFrame(metrics_history_keeper.history)
history

In [ ]:
best_point = history["simulation_limited/score_avg"].idxmax()
history.loc[best_point]

Let's see how the agent is doing in the real environment

In [ ]:
lightning_model.prepare_testing(num_episodes=2, limit_steps=None, return_unfinished=False)
metrics = trainer.test(lightning_model, torch.utils.data.DataLoader([0]))[0]

In [ ]:
agent = Agent(lightning_model.model, device, supports_batch=True)
score = run_episode(agent, show_progress=True, capture_video=True);
print(f"Score: {score:.2f}")

In [ ]:
wandb_logger.experiment.finish()

# DAGGER

Now we can implement DAgger, you have downloaded a relatively well trained model you can use as an expert for this purpose.

Load expert model that is provided as ONNX file.

## Load the expert

In [ ]:
# Load expert
expert_model = ConvertModel(onnx.load("expert.onnx"))

# Freeze expert weights
for p in expert_model.parameters():
    p.requires_grad_(False)
    
expert_agent = Agent(expert_model, device, supports_batch=False)

Next, you have to implement the DAgger algorithm (see slides for details). This function implements the core idea of DAgger:


1. Choose the policy with probability beta
2. Sample T-step trajectories using this policy
3. Label the gathered states with the expert

The aggregation and training part are already implemented.

In [ ]:
from __future__ import annotations

def collect_data_parallel(
    train_agent: Agent,
    expert_agent: Agent,
    num_parallel_envs: int,
    num_steps: int,
    follow_expert: bool,
    single_frame: bool = True,
) -> Iterable[tuple[np.ndarray, np.ndarray]]:
    """
    Yields:
      state: np.ndarray of shape observation_shape
      action: np.ndarray scalar
    in total it yields (num_parallel_envs * num_steps) times
    """

    env = gym.vector.AsyncVectorEnv([
        lambda: make_env(seed=None, capture_video=False)
        for _ in range(num_parallel_envs)
    ])
    
    if num_steps is None:
        num_steps = env.get_attr("spec")[0].max_episode_steps
    if num_steps is None:
        raise ValueError("num_steps must not be None")

    states, _ = env.reset()
    for i in tqdm(range(num_steps), desc="Collecting data - step"):
        expert_actions = expert_agent.select_action(states)
        if follow_expert:
            actions = expert_actions
        else:
            actions = train_agent.select_action(states)
        states, rewards, terminateds, truncateds, _ = env.step(actions)
        if single_frame:
            states_out = states[:, (-1,), :, :]
        else:
            states_out = states
        yield from zip(states_out, expert_actions)

    env.close()


In [ ]:
class DaggerDataCollectionCallback(lightning.Callback):
    def __init__(
        self,
        train_set: DemonstrationDataset,
        collect_every_nth_epoch: int,
        expert_agent: Agent,
        device: torch.device,
        num_parallel_envs: int,
        num_steps_follow_train: int,
        num_steps_follow_expert: int
    ) -> None:
        self.train_set = train_set
        self.collect_every_nth_epoch = collect_every_nth_epoch
        self.expert_agent = expert_agent
        self.device = device
        self.num_parallel_envs = num_parallel_envs
        self.num_steps_follow_train = num_steps_follow_train
        self.num_steps_follow_expert = num_steps_follow_expert
        self.duration_wall_time = 0
        self.duration_process_time = 0

    def setup(self, trainer: lightning.Trainer, lightning_model: lightning.LightningModule, stage: str) -> None:
        if trainer.reload_dataloaders_every_n_epochs == 0:
            raise ValueError(
                f"{self.__class__.__name__} requires"
                "`trainer.reload_dataloaders_every_n_epochs` to be a positive integer, ideally 1."
            )

    def on_train_epoch_start(self, trainer: lightning.Trainer, lightning_model: LightningPolicyNetwork) -> None:
        lightning_model.log_dict({
            "dagger/dataset_size": float(len(self.train_set)),
            "dagger/collect_data_wall_seconds": self.duration_wall_time,
            "dagger/collect_data_process_seconds": self.duration_process_time,
            "dagger/num_steps_follow_train": float(self.num_steps_follow_train),
            "dagger/num_steps_follow_expert": float(self.num_steps_follow_expert),
            "dagger/num_parallel_envs": float(self.num_parallel_envs),
        })

    def on_train_epoch_end(self, trainer: lightning.Trainer, lightning_model: LightningPolicyNetwork) -> None:
        if self.collect_every_nth_epoch is None:
            return
        if trainer.current_epoch % self.collect_every_nth_epoch != 0:
            return
        if trainer.current_epoch == trainer.max_epochs - 1:
            return

        train_agent = Agent(lightning_model.model, self.device, supports_batch=True)
        start_wall_time = time.time()
        start_process_time = time.process_time()

        self.train_set.append_iterable(
            collect_data_parallel(
                train_agent,
                self.expert_agent,
                self.num_parallel_envs,
                self.num_steps_follow_expert,
                follow_expert=True,
            )
        )

        self.train_set.append_iterable(
            collect_data_parallel(
                train_agent,
                self.expert_agent,
                self.num_parallel_envs,
                self.num_steps_follow_train,
                follow_expert=False,
            )
        )

        self.duration_wall_time = time.time() - start_wall_time
        self.duration_process_time = time.process_time() - start_process_time


Put everything together now. 
1. Create new samples using the DAgger algorithm
2. Continue training your agent
3. Export your fully trained agent as an ONNX file

In [ ]:

wandb_logger = lightning.pytorch.loggers.WandbLogger(
    project="jku-deep-rl_imitation-learning",
    save_dir="wandb",
)

checkpointing = lightning.pytorch.callbacks.ModelCheckpoint(
    monitor="simulation_limited/score_avg",
    mode="max",
    save_last=True,
    save_top_k=5,
    dirpath=f"checkpoints/{wandb_logger.experiment.name}/",
    filename="step={step}_score-avg={simulation/score_avg:.2f}_score-percentile-30={simulation/score_percentile_30}",
    auto_insert_metric_name=False,
)

metrics_history_keeper = MetricsHistoryKeeper()

model_config = dict(
    actions=action_mapping,
    architecture="efficientnet_b0",
    input_conv="features.0.0",
    output_lin="classifier.1",
)

lightning_model = LightningPolicyNetwork(
    model_config,
    num_val_episodes=8,
    limit_val_steps=500,
    label_smoothing=0.01,
    dummy_score_during_sanity_check=0,
)

datamodule = MyDataModule(
    train_set=train_set,
    val_set=val_set,
    num_workers=4,
    batch_size=128,
    prefetch_factor=8,
    pin_memory=True,
)

dagger_data_collection_callback = DaggerDataCollectionCallback(
    train_set=train_set,
    collect_every_nth_epoch=1,
    expert_agent=expert_agent,
    num_parallel_envs=4,
    num_steps_follow_train=1000,
    num_steps_follow_expert=3000,
)

wandb_logger.experiment.config.update({
    "dagger_enabled": True,
})

num_batches = len(train_set) / datamodule.batch_size

trainer = lightning.Trainer(
    logger=wandb_logger,
    val_check_interval=num_batches,
    reload_dataloaders_every_n_epochs=1,
    max_epochs=3,
    precision="16-mixed",
    accelerator="auto",
    callbacks=[
        checkpointing,
        metrics_history_keeper,
        dagger_data_collection_callback, # CRUCIAL !!!
    ],
)


In [ ]:
save_as_onnx(
    lightning_model.model.to("cpu"),
    sample_state.to("cpu"),
    logger.onnx_file
)

In [ ]:
agent = Agent(lightning_model.model, device, supports_batch=True)
scores = measure_scores_parallel(agent, num_episodes=32)
scores = np.array(list(scores))

In [ ]:
scores.mean(), scores.std()